In [48]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import plotly
import seaborn as sns
import plotly.express as px

Data cleaning

In [50]:
df=pd.read_csv('Zillow_NYC_Data.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
df['Lot']=df.HOA
df.Lot=[x if ('Acres' in str(x))==True else np.nan for x in df.HOA]
df.HOA=[x if ('Acres' in str(x))==False else np.nan for x in df.HOA]
df.Lot=[x if ('sqft' in str(x))==True else np.nan for x in df.HOA]
df.HOA=[x if ('sqft' in str(x))==False else np.nan for x in df.HOA]
df.loc[df['HOA']=='No Data','HOA']=np.nan
df.loc[df['Lot']=='No Data','Lot']=np.nan
df.HOA=[re.findall('\d{1,}\,{0,1}\d{0,}',x)[0].replace(',','') if type(x)==str else np.nan for x in df.HOA]
df.Lot=[re.findall('\d{1,}\,{0,1}\d{0,}',x)[0].replace(',','') if type(x)==str else np.nan for x in df.Lot]
df.loc[df['borough']=='Manhatan','borough']='Manhattan'
df.columns=df.columns.str.lower()
df=df.loc[df['bedrooms']!='-- bds,',:]
df.loc[df['bedrooms']=='Studio,','bedrooms']=0
df['bedrooms']=[float(x) for x in df['bedrooms']]
print(df.shape)

(792, 22)


In [51]:
df.dropna(subset=['price'],axis=0,inplace=True)
df.dropna(subset=['area'],axis=0,inplace=True)
df.dropna(subset=['construction'],axis=0,inplace=True)
df.drop('price_sqft',axis=1,inplace=True)
df=df.loc[df['construction']!='No Data',:]
df['price_area']=df['price']/df['area']
print(df.columns)

Index(['price', 'bedrooms', 'bathrooms', 'area', 'address', 'link', 'type',
       'construction', 'parking', 'heating', 'hoa', 'principal_interest',
       'mortgage', 'tax', 'insurance', 'school', 'ranking', 'latitude',
       'longitude', 'borough', 'lot', 'price_area'],
      dtype='object')


In [52]:
df.loc[:,'construction']=[int(x) for x in df.loc[:,'construction']]
df.loc[:,'price']=[int(x) for x in df.loc[:,'price']]
df.loc[:,'area']=[int(x) for x in df.loc[:,'area']]
df.loc[:,'age']=2021-df.loc[:,'construction']
df.loc[:,'latitude']=[float(x) for x in df.loc[:,'latitude']]
df.loc[:,'longitude']=[float(x) for x in df.loc[:,'longitude']]

In [53]:
df_temp=df.loc[:,['price']].describe()
df_temp.loc['cov','price']=df_temp.loc['std','price']/df_temp.loc['mean','price']
df_temp=df_temp.reset_index()
df_temp

,index,price
0,count,5.730000e+02
1,mean,1.423245e+06
2,std,2.102859e+06
3,min,8.900000e+04
4,25%,5.350000e+05
5,50%,8.390000e+05
6,75%,1.400000e+06
7,max,2.090000e+07
8,cov,1.477510e+00


In [54]:
df_temp=df.loc[:,['price']].describe()
df_temp.loc['cov','price']=df_temp.loc['std','price']/df_temp.loc['mean','price']
df_temp=df_temp.reset_index()
print(df_temp)
df_temp.to_csv('price_data.csv')
print('*'*100)
df_temp=df.groupby(['borough'])['price'].agg([np.min,np.max,np.mean,np.median,np.std,'count']).reset_index().sort_values('median')
df_temp.loc[:,'cov']=df_temp.loc[:,'std']/df_temp.loc[:,'mean']
print(df_temp)
df_temp.to_csv('price_borough.csv')
print('*'*100)
fig=px.histogram(df,x='price',nbins=20,title='Frequency for Price',labels={'price':'Price','count':'Frequency'})
fig.show()
#fig.write_image(fig, 'price_histogram.png')
print('*'*100)
fig=px.histogram(df,x='bedrooms',nbins=20,title='Frequency for Bedrooms',labels={'bedrooms':'Bedrooms','count':'Frequency'})
fig.show()
#fig.write_image(fig, 'bedrooms_histogram.png')
print('*'*100)
fig=px.histogram(df,x='bathrooms',nbins=20,title='Frequency for Bathrooms',labels={'bathrooms':'Bathrooms','count':'Frequency'})
fig.show()
#fig.write_image(fig, 'bathrooms_histogram.png')
print('*'*100)
df_temp=df.groupby(['borough'])['price_area'].agg([np.min,np.max,np.mean,np.median,np.std,'count']).reset_index().sort_values('median')
df_temp.loc[:,'cov']=df_temp.loc[:,'std']/df_temp.loc[:,'mean']
print(df_temp)
df_temp.to_csv('price_area_borough.csv')

   index         price
0  count  5.730000e+02
1   mean  1.423245e+06
2    std  2.102859e+06
3    min  8.900000e+04
4    25%  5.350000e+05
5    50%  8.390000e+05
6    75%  1.400000e+06
7    max  2.090000e+07
8    cov  1.477510e+00
****************************************************************************************************
         borough    amin      amax          mean  median           std  count  \
3         Queens  175000  20900000  1.333540e+06  765000  2.296484e+06    188   
4  Staten Island  179000  14700000  1.808535e+06  799900  2.815775e+06     43   
1       Brooklyn  101000   8500000  1.231028e+06  822000  1.319977e+06    122   
0          Bronx  207000   8450000  1.287068e+06  828000  1.480303e+06     41   
2      Manhattan   89000  16499000  1.613359e+06  950000  2.266414e+06    174   

        cov  
3  1.722096  
4  1.556937  
1  1.072256  
0  1.150135  
2  1.404779  
**************************************************************************************************

****************************************************************************************************


****************************************************************************************************


****************************************************************************************************
         borough        amin          amax         mean      median  \
0          Bronx    3.953699   2776.864936   704.051947  436.875000   
4  Staten Island  238.666667   2414.583333   837.217021  493.315508   
1       Brooklyn    6.287292   2742.049470   759.538571  538.178077   
3         Queens    1.287565   4984.497973   793.094629  626.428571   
2      Manhattan    3.204253  25692.307692  1045.887348  736.070912   

           std  count       cov  
0   560.011367     41  0.795412  
4   628.993313     43  0.751291  
1   575.182529    122  0.757279  
3   616.991148    188  0.777954  
2  1989.853689    174  1.902551  


"Queens" and "Manhattan" are selected for more investigation. They have the highest COV for price and price per area respectively. 

In [55]:
fig=px.scatter(df.loc[df['borough']=='Queens'],x='area',y='price',color='age',log_x=True,title='Price vs. Area/ Queens',
              labels={'area':'Area','price':'Price'})
fig.show()
fig=px.scatter(df.loc[df['borough']=='Manhattan'],x='area',y='price',color='age',log_x=True,title='Price vs. Area/ Manhattan',
              labels={'area':'Area','price':'Price'})
fig.show()

In [56]:
fig=px.scatter(df.loc[df['borough']=='Queens'],x='age',y='price_area',color='bedrooms',title='Price_Area for Queens',
              labels={'area':'Area','price':'Price'})
fig.show()
fig=px.scatter(df.loc[df['borough']=='Manhattan'],x='age',y='price_area',color='bedrooms',title='Price_Area for Queens',
              labels={'area':'Area','price':'Price'})
fig.show()

In [57]:
df_queens=df.loc[df['borough']=='Queens',:]
df_manhattan=df.loc[df['borough']=='Manhattan',:]
df_temp=df_queens.corr()
print(df_temp)
df_temp.to_csv('Queens_Correlation.csv')
df_temp=df_manhattan.corr()
print(df_temp)
df_temp.to_csv('Manhattan_Correlation.csv')

                 price  bedrooms  bathrooms      area  construction  latitude  \
price         1.000000  0.326836   0.595211 -0.004569      0.022383  0.087603   
bedrooms      0.326836  1.000000   0.776337 -0.116269     -0.131533 -0.031802   
bathrooms     0.595211  0.776337   1.000000 -0.085248     -0.050659  0.002908   
area         -0.004569 -0.116269  -0.085248  1.000000     -0.044199  0.039917   
construction  0.022383 -0.131533  -0.050659 -0.044199      1.000000 -0.047004   
latitude      0.087603 -0.031802   0.002908  0.039917     -0.047004  1.000000   
longitude     0.010232  0.070736   0.026339 -0.003240     -0.102462 -0.253538   
price_area    0.671974 -0.061069   0.206826 -0.211604      0.264247  0.083952   
age          -0.022383  0.131533   0.050659  0.044199     -1.000000  0.047004   

              longitude  price_area       age  
price          0.010232    0.671974 -0.022383  
bedrooms       0.070736   -0.061069  0.131533  
bathrooms      0.026339    0.206826  0.050659

In [58]:
fig=px.histogram(df_queens,x=['bedrooms'],title='Number of Bedrooms for Queens',labels={'value':'Bedrooms','count ':'Count'})
fig.show()
fig=px.histogram(df_manhattan,x='bedrooms',title='Number of Bedrooms for Manhattan',labels={'bedrooms':'Bedrooms','count':'Count'})
fig.show()

In [59]:
fig=px.histogram(df_queens,x='bathrooms',title='Number of Bathrooms for Queens',labels={'value':'Bedrooms','count ':'Count'})
fig.show()
fig=px.histogram(df_manhattan,x='bathrooms',title='Number of Bathrooms for Manhattan',labels={'bedrooms':'Bedrooms','count':'Count'})
fig.show()

In [60]:
df_queens.bedrooms=[int(x) for x in df_queens['bedrooms']]
fig=px.scatter(df_queens,x='area',y='price',color='age',size='bedrooms',log_x=True,log_y=True)
fig.show()
df_manhattan.bedrooms=[int(x) for x in df_manhattan['bedrooms']]
fig=px.scatter(df_queens,x='area',y='price',color='age',size='bedrooms',log_x=True,log_y=True)
fig.show()

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5168: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [62]:
df_queens_b2=df_queens.loc[df_queens['bedrooms']==2,:]
fig=px.scatter(df_queens_b2,x='area',y='price',size='bathrooms',color='age',log_x=True,log_y=True,title='Price vs. Area for Queens/ Bedrooms=2')
fig.show()
df_queens_b2=df_queens.loc[df_queens['bedrooms']==2,:]
fig=px.scatter(df_queens_b2,x='age',y='price_area',color='bathrooms',log_x=True,log_y=True,title='Price_Area vs. Age for Queens/ Bedrooms=2')
fig.show()

In [63]:
print(df_queens.corr())

                 price  bedrooms  bathrooms      area  construction  latitude  \
price         1.000000  0.326836   0.595211 -0.004569      0.022383  0.087603   
bedrooms      0.326836  1.000000   0.776337 -0.116269     -0.131533 -0.031802   
bathrooms     0.595211  0.776337   1.000000 -0.085248     -0.050659  0.002908   
area         -0.004569 -0.116269  -0.085248  1.000000     -0.044199  0.039917   
construction  0.022383 -0.131533  -0.050659 -0.044199      1.000000 -0.047004   
latitude      0.087603 -0.031802   0.002908  0.039917     -0.047004  1.000000   
longitude     0.010232  0.070736   0.026339 -0.003240     -0.102462 -0.253538   
price_area    0.671974 -0.061069   0.206826 -0.211604      0.264247  0.083952   
age          -0.022383  0.131533   0.050659  0.044199     -1.000000  0.047004   

              longitude  price_area       age  
price          0.010232    0.671974 -0.022383  
bedrooms       0.070736   -0.061069  0.131533  
bathrooms      0.026339    0.206826  0.050659

In [64]:
print(df_manhattan.corr())

                 price  bedrooms  bathrooms      area  construction  latitude  \
price         1.000000  0.406599   0.531989  0.073914      0.126819  0.024072   
bedrooms      0.406599  1.000000   0.826771  0.036211     -0.201217 -0.045353   
bathrooms     0.531989  0.826771   1.000000  0.012192      0.024121 -0.042120   
area          0.073914  0.036211   0.012192  1.000000      0.015243  0.012864   
construction  0.126819 -0.201217   0.024121  0.015243      1.000000 -0.136835   
latitude      0.024072 -0.045353  -0.042120  0.012864     -0.136835  1.000000   
longitude    -0.044121 -0.075687  -0.068363 -0.013135     -0.018271  0.053745   
price_area    0.125909 -0.059961  -0.068703 -0.057136      0.077621 -0.037213   
age          -0.126819  0.201217  -0.024121 -0.015243     -1.000000  0.136835   

              longitude  price_area       age  
price         -0.044121    0.125909 -0.126819  
bedrooms      -0.075687   -0.059961  0.201217  
bathrooms     -0.068363   -0.068703 -0.024121

In [65]:
df_manhattan_b2=df_manhattan.loc[df_manhattan['bedrooms']==2,:]
fig=px.scatter(df_manhattan_b2,x='area',y='price',size='age',color='bathrooms',log_x=True,log_y=True)
fig.show()
df_manhattan_b2=df_manhattan.loc[df_manhattan['bedrooms']==2,:]
fig=px.scatter(df_manhattan_b2,x='age',y='price_area',color='bathrooms',log_x=True,log_y=True)
fig.show()

In [ ]:
df_temp=df_manhattan.groupby('bedrooms')['price','price_area','area','age'].agg(['min','max','mean','median'])[['price_area','age']].reset_index()
print(df_temp)

In [ ]:
fig=px.scatter(df_temp,x=df_temp.bedrooms,y=df_temp.price_area.median)
fig.show()
#fig=px.box(df_manhattan,x='bedrooms',y='price')
#fig.show()
#fig=px.scatter(df_temp,x='bedrooms',y='median')
#fig.show()